# Toy Example - Tabularized GiGL

Latest version of this notebook can be found on [github](https://github.com/Snapchat/GiGL/blob/main/examples/toy_visual_example/toy_example_walkthrough.ipynb)


This notebook provides a walkthrough of preprocessing, subgraph sampling, and split generation components with a small toy graph for GiGL's Tabularized setting for training/inference. It will help you understand how each of these components prepare tabularized subgraphs.


## Overview Of Components
This notebook demonstrates the process of a simple, human-digestible graph being passed through all the pipeline components in GiGL in preparation for training to help understand how each of the components work.

The pipeline consists of the following components:

- **Config Populator**: Takes a template config and creates a frozen workflow config that dictates all inputs/outputs and business parameters that are read and used by each subsequent component.
    - Input: `template_config.yaml`
    - Output: `frozen_gbml_config.yaml`
&nbsp;

- **Data Preprocessor**: Transforms necessary node and edge feature assets as needed as a precursor step in most ML tasks according to the user-provided data preprocessor config class.
    - Input: `frozen_gbml_config.yaml`, which includes the user-defined preprocessor class for custom logic, and custom arguments can be passed under dataPreprocessorArgs.
    - Output: PreprocessedMetadata Proto, which includes inferred GraphMetadata and preprocessed graph data Tfrecords after applying the user-defined preprocessing function.
&nbsp;

- **Subgraph Sampler**: Samples k-hop subgraphs for each node according to user-provided arguments.
    - Input: `frozen_gbml_config.yaml`, `resource_config.yaml`
    - Output: Subgraph Samples (tfrecord format based on predefined schema in protos) are stored in the URI defined in the flattenedGraphMetadata field.
&nbsp;

- **Split Generator**: Splits subgraph sampler outputs into train/test/val sets according to the user-provided split strategy class.
    - Input: `frozen_gbml_config.yaml`, which includes an instance of SplitStrategy and an instance of Assigner.
    - Output: TFRecord samples.
&nbsp;

- **Trainer**: The trainer component reads the output of the split generator and trains a model on the training set, stops based on the validation set, and evaluates on the test set.
    - Input: `frozen_gbml_config.yaml`
    - Output: state_dict stored in trainedModelUri.
&nbsp;

- **Inferencer**: Runs inference of a trained model on samples generated by the Subgraph Sampler.
    - Input: `frozen_gbml_config.yaml`
    - Output: Embeddings and/or prediction assets.
&nbsp;


In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Silence TF logspam

from gigl.common.utils.jupyter_magics import change_working_dir_to_gigl_root
change_working_dir_to_gigl_root()

## Input Graph

We use the input graph defined in [examples/toy_visual_example/graph_config.yaml](./graph_config.yaml). 
You are welcome to change this file to a custom graph of your own choosing.

### Visualizing the Input Graph

In [ ]:
from torch_geometric.data import HeteroData

from gigl.common.utils.jupyter_magics import GraphVisualizer
from gigl.src.mocking.toy_asset_mocker import load_toy_graph


original_graph_heterodata: HeteroData = load_toy_graph(graph_config_path="examples/toy_visual_example/graph_config.yaml")
# Visualize the graph
GraphVisualizer.visualize_graph(original_graph_heterodata)

### Setting up Configs

The first thing we need to do is create the resource and task configs. 

- **Task Config**: Specifies task-related configurations, guiding the behavior of components according to the needs of your machine learning task. See [Task Config Guide](../../docs/user_guide/config_guides/task_config_guide.md). For this task, we have already provided a task config: [task_config.yaml](./task_config.yaml).

- **Resource Config**: Details the resource allocation and environmental settings across all GiGL components. This encompasses shared resources for all components, as well as component-specific settings. See [Resource Config Guide](../../docs/user_guide/config_guides/resource_config_guide.md). For this task, we provide a resource [resource_config.yaml](./resource_config.yaml). The provided default values in `shared_resource_config.common_compute_config` will need to be changed.

  - **Instructions to configure the resource config to work**:
    If you have not already, please follow the [Quick Start Guide](../../docs/user_guide/getting_started/quick_start.md) to set up your cloud environment and create a default test resource config. You can then copy the relevant `shared_resource_config.common_compute_config` to [resource_config.yaml](./resource_config.yaml).

In [ ]:
import datetime
import getpass
import pathlib
import textwrap


from gigl.common import Uri, UriFactory
notebook_dir = pathlib.Path("./examples/toy_visual_example").as_posix() # We should be in root dir because of cell # 1
# Add the root gigl dir to the Python path so `example` folder can be imported as a module.

# You are welcome to customize these  to point to your own configuration files.
JOB_NAME = f"{getpass.getuser()}_gigl_toy_example_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
TEMPLATE_TASK_CONFIG_PATH: Uri = UriFactory.create_uri(f"{notebook_dir}/template_task_config.yaml")
FROZEN_TASK_CONFIG_POINTER_FILE_PATH: Uri = UriFactory.create_uri(f"/tmp/GiGL/{JOB_NAME}/frozen_task_config.yaml")
pathlib.Path(FROZEN_TASK_CONFIG_POINTER_FILE_PATH.uri).parent.mkdir(parents=True, exist_ok=True)
# Ensure you change the resource config path to point to your own resource configuration
# i.e. what was exported to $GIGL_TEST_DEFAULT_RESOURCE_CONFIG as part of the quick start guide.
RESOURCE_CONFIG_PATH: Uri = UriFactory.create_uri(os.environ.get("GIGL_TEST_DEFAULT_RESOURCE_CONFIG", f"{notebook_dir}/resource_config.yaml"))

# Export string format of the uris so we can reference them in cells that execute bash commands below.
os.environ["TEMPLATE_TASK_CONFIG_PATH"] = TEMPLATE_TASK_CONFIG_PATH.uri
os.environ["FROZEN_TASK_CONFIG_POINTER_FILE_PATH"] = FROZEN_TASK_CONFIG_POINTER_FILE_PATH.uri
os.environ["RESOURCE_CONFIG_PATH"] = RESOURCE_CONFIG_PATH.uri

print(f"JOB_NAME: {JOB_NAME}")
print(f"TEMPLATE_TASK_CONFIG_PATH: {TEMPLATE_TASK_CONFIG_PATH.uri}")
print(f"FROZEN_TASK_CONFIG_POINTER_FILE_PATH: {FROZEN_TASK_CONFIG_POINTER_FILE_PATH.uri}")
print(f"RESOURCE_CONFIG_PATH: {RESOURCE_CONFIG_PATH.uri}")

### Note on the Use of Mocked Assets

This step is already done for you. We provide instructions below for posterity, in case the mocked data input ["graph_config.yaml"](./graph_config.yaml) is updated.

You can choose to update `MOCK_DATA_GCS_BUCKET` and `MOCK_DATA_BQ_DATASET_NAME` in `python/gigl/src/mocking/lib/constants.py` to upload to resources you own.

We run the following command to upload the relevant mocks to GCS and BQ:
```bash
python -m gigl.src.mocking.dataset_asset_mocking_suite \
--select mock_toy_graph_homogeneous_node_anchor_based_link_prediction_dataset \
--resource_config_uri=examples/toy_visual_example/resource_config.yaml
```

Subsequently, we can update the paths in [task_config.yaml](./task_config.yaml).

## Validating the Configs

We provide the ability to validate your resource and task configs. Although the validation is not exhaustive, it does help assert that the more common issues are not present before expensive compute is scheduled.

In [ ]:
from gigl.src.validation_check.config_validator import kfp_validation_checks

validator = kfp_validation_checks(
    job_name=JOB_NAME,
    task_config_uri=TEMPLATE_TASK_CONFIG_PATH,
    resource_config_uri=RESOURCE_CONFIG_PATH,
    start_at="config_populator",
)

### Config Populator

Takes in a template `GbmlConfig` and outputs a frozen `GbmlConfig` by populating all job-related metadata paths in `sharedConfig`. These are mostly GCS paths that the following components read and write from, and use as an intermediary data communication medium. For example, the field `sharedConfig.trainedModelMetadata` is populated with a GCS URI, which indicates to the Trainer to write the trained model to this path, and to the Inferencer to read the model from this path. See the full [Config Populator Guide](../../docs/user_guide/overview/components/config_populator.md).

After running the command below, we will have created a frozen config and uploaded it to the `perm_assets_bucket` provided in the `resource config`. The path to that file will be stored in the file at `FROZEN_TASK_CONFIG_POINTER_FILE_PATH`.

In [ ]:
%%capture
# We suppress the output of this cell to avoid cluttering the notebook with logs. Remove %%%capture if you want to see the output.

!python -m \
    gigl.src.config_populator.config_populator \
    --job_name="$JOB_NAME" \
    --template_uri="$TEMPLATE_TASK_CONFIG_PATH" \
    --resource_config_uri="$RESOURCE_CONFIG_PATH" \
    --output_file_path_frozen_gbml_config_uri="$FROZEN_TASK_CONFIG_POINTER_FILE_PATH"

In [ ]:
# The command above will write the frozen task config path to the file specified by `FROZEN_TASK_CONFIG_POINTER_FILE_PATH`.
# Lets see where it was generated
FROZEN_TASK_CONFIG_PATH: Uri
with open(FROZEN_TASK_CONFIG_POINTER_FILE_PATH.uri, 'r') as file:
    FROZEN_TASK_CONFIG_PATH = UriFactory.create_uri(file.read().strip())
print(f"FROZEN_TASK_CONFIG_PATH: {FROZEN_TASK_CONFIG_PATH}")

## Visualizing the Diff Between Template and Frozen Config

We now have a frozen task config, with the path specified by `FROZEN_TASK_CONFIG_PATH`. We visualize the diff between the `frozen_task_config` generated by the `config_populator` and the original `template_task_config`. All the code below is just to do that and has nothing to do with GiGL.

Specifically, note that:
1. The component added `sharedConfig` to the YAML, which contains all the intermediary and final output paths for each component.
2. It also added a `condensedEdgeTypeMap` and a `condensedNodeTypeMap`, which map all provided edge types and node types to `int` to save storage space:
   - `EdgeType: Tuple[srcNodeType: str, relation: str, dstNodeType: str)] -> int`, and 
   - `NodeType: str -> int`
   - Note: You may also provide your own condensedMaps; they will be generated for you if not provided.

In [ ]:
from gigl.common.utils.jupyter_magics import show_task_config_colored_unified_diff

show_task_config_colored_unified_diff(
    f1_uri=FROZEN_TASK_CONFIG_PATH,
    f2_uri=TEMPLATE_TASK_CONFIG_PATH,
    f1_name='frozen_task_config.yaml',
    f2_name='template_task_config.yaml'
)

In [ ]:
# We will load the frozen task and resource configs file into an object so we can reference it in the next cells
from gigl.env.pipelines_config import get_resource_config
from gigl.src.common.types.pb_wrappers.gbml_config import GbmlConfigPbWrapper
from gigl.src.common.types.pb_wrappers.gigl_resource_config import GiglResourceConfigWrapper

frozen_task_config = GbmlConfigPbWrapper.get_gbml_config_pb_wrapper_from_uri(
    gbml_config_uri=FROZEN_TASK_CONFIG_PATH
)
resource_config: GiglResourceConfigWrapper = get_resource_config(
    resource_config_uri=RESOURCE_CONFIG_PATH
)

## Building Scala Jars

GiGL uses Spark/Scala to run data processing pipelines for the [subgraph sampler](../../docs/user_guide/overview/components/subgraph_sampler.md) and [split generator](../../docs/user_guide/overview/components/split_generator.md) components.

As such, we need to build and packages jars so that we can run the pipelines.

In [ ]:
from scripts.scala_packager import ScalaPackager
sgs_spark31, sgs_spark35, split_gen  = ScalaPackager().package_all()

print(f"Subgraph Sampler Spark 3.1 jar: {sgs_spark31.uri}")
print(f"Subgraph Sampler Spark 3.5 jar: {sgs_spark35.uri}")
print(f"Split Generator Spark 3.5 jar: {split_gen.uri}")

## Compiling Source Docker Images

You will need to build and push Docker images with your custom code so that individual GiGL components can leverage your code. For this experiment, we will consider the toy_visual_example specs and relevant code to be "custom code," and we will guide you on how to build a Docker image with the code.

We will make use of `scripts/build_and_push_docker_image.py` for this.

Note that this builds `containers/Dockerfile.src` and `containers/Dockerfile.dataflow.src`, which have instructions to `COPY` the `examples` folder - which contains all the source code for this example, and it has all the GiGL source code.

In [ ]:
%%capture
# We suppress the output of this cell to avoid cluttering the notebook with logs. Remove %%%capture if you want to see the output.

from scripts.build_and_push_docker_image import build_and_push_cpu_image, build_and_push_cuda_image, build_and_push_dataflow_image
import datetime

project = resource_config.project
curr_datetime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Change to whereever you want to store the docker images.
DOCKER_ARTIFACT_REGISTRY = f"us-central1-docker.pkg.dev/{project}/gigl-base-images"

DOCKER_IMAGE_DATAFLOW_RUNTIME_NAME_WITH_TAG = f"{DOCKER_ARTIFACT_REGISTRY}/gigl_dataflow_runtime:{curr_datetime}"
DOCKER_IMAGE_MAIN_CUDA_NAME_WITH_TAG = f"{DOCKER_ARTIFACT_REGISTRY}/gigl_cuda:{curr_datetime}"
DOCKER_IMAGE_MAIN_CPU_NAME_WITH_TAG = f"{DOCKER_ARTIFACT_REGISTRY}/gigl_cpu:{curr_datetime}"
os.environ["DOCKER_IMAGE_DATAFLOW_RUNTIME_NAME_WITH_TAG"] = DOCKER_IMAGE_DATAFLOW_RUNTIME_NAME_WITH_TAG
os.environ["DOCKER_IMAGE_MAIN_CUDA_NAME_WITH_TAG"] = DOCKER_IMAGE_MAIN_CUDA_NAME_WITH_TAG
os.environ["DOCKER_IMAGE_MAIN_CPU_NAME_WITH_TAG"] = DOCKER_IMAGE_MAIN_CPU_NAME_WITH_TAG

build_and_push_dataflow_image(
    image_name=DOCKER_IMAGE_DATAFLOW_RUNTIME_NAME_WITH_TAG,
)
build_and_push_cuda_image(
    image_name=DOCKER_IMAGE_MAIN_CUDA_NAME_WITH_TAG,
)
build_and_push_cpu_image(
    image_name=DOCKER_IMAGE_MAIN_CPU_NAME_WITH_TAG,
)

print(f"""We built and pushed the following docker images:
- {DOCKER_IMAGE_DATAFLOW_RUNTIME_NAME_WITH_TAG}
- {DOCKER_IMAGE_MAIN_CUDA_NAME_WITH_TAG}
- {DOCKER_IMAGE_MAIN_CPU_NAME_WITH_TAG}
""")

## Data Preprocessor

Once we have a `frozen_task_config`, the first step is to preprocess the data.

The Data Preprocessor component uses [Tensorflow Transform](https://www.tensorflow.org/tfx/transform/get_started) to achieve data transformation in a distributed fashion. 

- Any custom preprocessing is to be defined in the preprocessor class, specified in the task config by `datasetConfig.dataPreprocessorConfig.dataPreprocessorConfigClsPath`.
- This class must inherit from {py:class}`gigl.src.data_preprocessor.lib.data_preprocessor_config.DataPreprocessorConfig`.

In your preprocessor spec, you must implement the following 3 functions as defined by the base class `DataPreprocessorConfig`:
  - `prepare_for_pipeline`: Preparing datasets for ingestion and transformation.
  - `get_nodes_preprocessing_spec`: Defining transformation imperatives for different node types.
  - `get_edges_preprocessing_spec`: Defining transformation imperatives for different edge types.

Please take a look at [toy_data_preprocessor_config.py](./toy_data_preprocessor_config.py) to see how these are defined. You will note that in this case, we are not doing anything special (i.e., no feature engineering), just reading from BQ and passing through the features. We could, if we wanted, define our own [preprocessing function](https://www.tensorflow.org/tfx/transform/get_started#preprocessing_function_example), and replace it with `build_passthrough_transform_preprocessing_fn()` defined in the code.

### Input Parameters and Output Paths for Data Preprocessor
Let's take a quick look at what these are from our frozen config.

In [ ]:

print("Frozen Config DataPreprocessor Information:")

print("- Data Preprocessor Config: Specifies what class to use for datapreprocessing and any arguments that might be passed in at runtime to that class")
print(textwrap.indent(str(frozen_task_config.dataset_config.data_preprocessor_config), '\t'))
print("- Preprocessed Metadata Uri: Specifies path to the preprocessed metadata file that will be generated by this component and used by subsequent components to understand and find the data that was preprocessed")
print(textwrap.indent(str(frozen_task_config.shared_config.preprocessed_metadata_uri), '\t'))

### Running Data Preprocessor and visualizing the Preprocessed Metadata

In [ ]:
%%capture

!python -m gigl.src.data_preprocessor.data_preprocessor \
--job_name=$JOB_NAME \
--task_config_uri=$FROZEN_TASK_CONFIG_PATH \
--resource_config_uri=$RESOURCE_CONFIG_PATH \
--custom_worker_image_uri=$DOCKER_IMAGE_DATAFLOW_RUNTIME_NAME_WITH_TAG

#### Upon completion of job, we will see the preprocessed metadata be populated

In [ ]:
frozen_task_config = GbmlConfigPbWrapper.get_gbml_config_pb_wrapper_from_uri(
    gbml_config_uri=FROZEN_TASK_CONFIG_PATH
)
preprocessed_metadata_pb = frozen_task_config.preprocessed_metadata_pb_wrapper.preprocessed_metadata_pb
print(preprocessed_metadata_pb)

You do not have to worry about these details in code as it is all handled by the data preprocessor component and subsequent data loaders
But, for the sake of understanding, we will investigate the condensed_node_type = 0 and condensed_edge_type = 0
If you remember the from the frozen config the mappings were as follows:

In [ ]:
print("Condensed Node Type Mapping:")
print(textwrap.indent(str(frozen_task_config.graph_metadata.condensed_node_type_map), '\t'))
print("Condensed Edge Type Mapping:")
print(textwrap.indent(str(frozen_task_config.graph_metadata.condensed_edge_type_map), '\t'))

preprocessed_nodes = preprocessed_metadata_pb.condensed_node_type_to_preprocessed_metadata[0].tfrecord_uri_prefix
preprocessed_edges = preprocessed_metadata_pb.condensed_edge_type_to_preprocessed_metadata[0].main_edge_info.tfrecord_uri_prefix
print(f"Preprocessed Nodes are stored in: {preprocessed_nodes}")
print(f"Preprocessed Edges are stored in: {preprocessed_edges}")

There is not a lot of data so we will have likely just generated one file for each of the preprocessed nodes and edges.


In [ ]:
!gsutil ls $preprocessed_nodes && gsutil ls $preprocessed_edges

### Subgraph Sampler

The Subgraph Sampler receives node and edge data from the Data Preprocessor and generates k-hop localized subgraphs for each node in the graph. The purpose is to store the neighborhood of each node independently, thereby reducing the memory footprint for downstream components, as they need not load the entire graph into memory but only batches of these node neighborhoods. 

To run the subgraph sampler, we use the following command:

In [ ]:
%%capture

!python -m gigl.src.subgraph_sampler.subgraph_sampler \
--job_name=$JOB_NAME \
--task_config_uri=$FROZEN_TASK_CONFIG_PATH \
--resource_config_uri=$RESOURCE_CONFIG_PATH

Upon completion, there will be two different directories of subgraph samples. One is the main node anchor-based link prediction samples, and the other is random negative rooted neighborhood samples, which are stored in the locations specified in the frozen_config:

In [ ]:
flattened_graph_metadata = frozen_task_config.shared_config.flattened_graph_metadata
print(flattened_graph_metadata)

The main k-hop node_anchor_based_link_prediction_samples include root nodes' neighborhoods, positive nodes' neighborhoods, and positive edges. These samples will be used for training.

The k-hop random_negative_rooted_neighborhood_samples (which include root nodes' neighborhoods) serve a dual purpose: they will be used for the inferencer and as random negative samples for training.

The random negatives are used for the model to learn non-existent (negative) edges since it could overfit on just positive samples. This means it would fail to generalize well to unseen data. The negative edges are just edges chosen at random. At a large scale, this would most probably be a negative edge. 

Below, we visualize the Root Node Neighborhood of 5, the Root Node Neighborhood of its pos_edge's destination node (1), and the resulting sample for root node 5. 

Since the subgraph sampler is sampling randomly here, you will get different subgraphs every time you run this.

For the purposes of example, we also provide some screenshots of what these graphs might look like:

When training, you may see a sample for node 9 as follows. Specifically, note that edge `9 --> 7` is classified as a positive edge, where SGS has sampled a 2-hop subgraph with incoming edges to both nodes 9 and 7.

<img src="./assets/link_pred_sample_node_9.png" alt="Link Pred Sample" width="50%" />

Secondly, we may choose to randomly sample a rooted node neighborhood to act as a "negative sample" (i.e., in this case, we sample node 1 and can have `9 --> 1` edge be a "negative sample").

<img src="./assets/rooted_neighborhood_node_1.png" alt="Random Negative Sample" width="50%" />






In [ ]:
from snapchat.research.gbml import training_samples_schema_pb2

# We will sample node with this id and visualize it. You will see positive edge marked in red and the root node with a black border.
SAMPLE_NODE_ID = 9
# We will sample random negative nodes with these ids and visualize them. You will see the root node with a black border.
SAMPLE_RANDOM_NEGATIVE_NODE_IDS = [1, 3]


from gigl.common.utils.jupyter_magics import GraphVisualizer, find_node_pb


print(f"The original global graph:")
GraphVisualizer.visualize_graph(original_graph_heterodata)


sample = find_node_pb(
    tfrecord_uri_prefix=flattened_graph_metadata.node_anchor_based_link_prediction_output.tfrecord_uri_prefix,
    node_id=SAMPLE_NODE_ID,
    pb_type=training_samples_schema_pb2.NodeAnchorBasedLinkPredictionSample
)
print(f"Node anchor prediction sample for node {SAMPLE_NODE_ID}:")
GraphVisualizer.plot_graph(sample)
for random_negative_node_id in SAMPLE_RANDOM_NEGATIVE_NODE_IDS:
    random_negative_sample = find_node_pb(
        tfrecord_uri_prefix=flattened_graph_metadata.node_anchor_based_link_prediction_output.node_type_to_random_negative_tfrecord_uri_prefix["user"],
        node_id=random_negative_node_id,
        pb_type=training_samples_schema_pb2.RootedNodeNeighborhood
    )
    print(f"Random negative sample for node {random_negative_node_id}:")
    GraphVisualizer.plot_graph(random_negative_sample)



### Split Generator

The Split Generator reads localized subgraph samples produced by the Subgraph Sampler and executes the user-specified split strategy logic to split the data into training, validation, and test sets. Several standard configurations of SplitStrategy and corresponding Assigner classes are already implemented at a GiGL platform level: transductive node classification, inductive node classification, and transductive link prediction split routines. For more information on split strategies in Graph Machine Learning, check out these resources:

1. [CS224W Lecture Slides](http://web.stanford.edu/class/cs224w/slides/07-theory.pdf)
2. [Graph Link Prediction](https://zqfang.github.io/2021-08-12-graph-linkpredict/) (relevant for explaining transductive vs inductive).

In this example, we are using the transductive strategy as specified in our frozen_config:

In [ ]:
print(frozen_task_config.dataset_config.split_generator_config)

For transductive, at training time, it uses training message edges to predict training supervision edges. At validation time, the training message edges and training supervision edges are used to predict the validation edges and then all 3 are used to predict test edges. Below is the command to run split generator:


In [ ]:
%%capture

!python -m gigl.src.split_generator.split_generator \
--job_name=$JOB_NAME \
--task_config_uri=$FROZEN_TASK_CONFIG_PATH \
--resource_config_uri=$RESOURCE_CONFIG_PATH

Upon completion, there will be 3 folders for train,test, and val. Each of them contains the protos for the positive and negaitve samples. The path for these folders is specified in the following location in the frozen_config:

In [ ]:
dataset_metadata = frozen_task_config.shared_config.dataset_metadata
print(dataset_metadata)


We can visualize the train,test, and val samples for all nodes.
Note, for this specific task config setup, not all val amd test samples will have positive labels. This is because edges are randomly assigned into "train", "val", or "test" buckets independent of whether or not they are supervision edges. Thus, although at scale this setting is okay i.e. with large data and a large batch size each batch will have some supervision edges, it can be a case that certain batches dont have any supervision edges. Thus, your val/test loops and early stopping logic may need to be carefully designed.

In [ ]:
for node_id in range(original_graph_heterodata.num_nodes):
    print(f"Node anchor prediction sample for node {node_id}:")
    sample_train = find_node_pb(
        tfrecord_uri_prefix=dataset_metadata.node_anchor_based_link_prediction_dataset.train_main_data_uri,
        node_id=node_id,
        pb_type=training_samples_schema_pb2.NodeAnchorBasedLinkPredictionSample
    )
    sample_val = find_node_pb(
        tfrecord_uri_prefix=dataset_metadata.node_anchor_based_link_prediction_dataset.val_main_data_uri,
        node_id=node_id,
        pb_type=training_samples_schema_pb2.NodeAnchorBasedLinkPredictionSample
    )
    sample_test = find_node_pb(
        tfrecord_uri_prefix=dataset_metadata.node_anchor_based_link_prediction_dataset.test_main_data_uri,
        node_id=node_id,
        pb_type=training_samples_schema_pb2.NodeAnchorBasedLinkPredictionSample
    )
    if sample_train:
        print(f"Train sample for node {node_id}: ")
        GraphVisualizer.plot_graph(sample_train)
    else:
        print(f"No train sample found for node {node_id}.")
    if sample_val:
        print(f"Validation sample for node {node_id}:")
        GraphVisualizer.plot_graph(sample_val)
    else:
        print(f"No validation sample found for node {node_id}.")
    if sample_test:
        print(f"Test sample for node {node_id}:")
        GraphVisualizer.plot_graph(sample_test)
    else:
        print(f"No test sample found for node {node_id}.")



At this point, we have our graph data samples ready to be processed by the trainer and inferencer components. These components will extract representations/embeddings by learning contextual information for the specified task.

## Training and Inference

Tabularized GiGL uses a config-based approach to training and inference. They use the [`GbmlConfig.TrainerConfig`](https://github.com/Snapchat/GiGL/blob/97fdb6e5d41f567900d729e9bb894a8f727dbd0c/proto/snapchat/research/gbml/gbml_config.proto#L176-L191) and [`GbmlConfig.InferencerConfig`](https://github.com/Snapchat/GiGL/blob/97fdb6e5d41f567900d729e9bb894a8f727dbd0c/proto/snapchat/research/gbml/gbml_config.proto#L193-L208) Proto messages.

In this case, they both use the [`NodeAnchorBasedLinkPredictionModelingTaskSpec`](https://github.com/Snapchat/GiGL/blob/97fdb6e5d41f567900d729e9bb894a8f727dbd0c/python/gigl/src/common/modeling_task_specs/node_anchor_based_link_prediction_modeling_task_spec.py#L66)

You can run the Trainer and Inferencer the same as the other components

In [ ]:
# Copy over resource config to GCS so that training and inference can access it.
# If we are already using a GCS URI, we will just use it as is.
from gigl.common import GcsUri, LocalUri
from gigl.src.common.utils.file_loader import FileLoader

if isinstance(RESOURCE_CONFIG_PATH, LocalUri):
    GCS_RESOURCE_CONFIG_PATH = GcsUri.join(resource_config.temp_assets_regional_bucket_path, JOB_NAME, "resource_config.yaml")
    loader = FileLoader()
    loader.load_file(
        file_uri_src=RESOURCE_CONFIG_PATH,
        file_uri_dst=GCS_RESOURCE_CONFIG_PATH
    )
else:
    GCS_RESOURCE_CONFIG_PATH = RESOURCE_CONFIG_PATH
print(f"GCS_RESOURCE_CONFIG_PATH: {GCS_RESOURCE_CONFIG_PATH}")

In [ ]:
%%bash
echo "JOB_NAME: $JOB_NAME"
echo "FROZEN_TASK_CONFIG_PATH: $FROZEN_TASK_CONFIG_PATH"
echo "GCS_RESOURCE_CONFIG_PATH: $GCS_RESOURCE_CONFIG_PATH"
echo "DOCKER_IMAGE_MAIN_CPU_NAME_WITH_TAG: $DOCKER_IMAGE_MAIN_CPU_NAME_WITH_TAG"
echo "DOCKER_IMAGE_MAIN_CUDA_NAME_WITH_TAG: $DOCKER_IMAGE_MAIN_CUDA_NAME_WITH_TAG"
echo "DOCKER_IMAGE_DATAFLOW_RUNTIME_NAME_WITH_TAG: $DOCKER_IMAGE_DATAFLOW_RUNTIME_NAME_WITH_TAG"

python -m gigl.src.training.trainer \
--job_name=$JOB_NAME \
--task_config_uri=$FROZEN_TASK_CONFIG_PATH \
--resource_config_uri=$GCS_RESOURCE_CONFIG_PATH \
--cpu_docker_uri=$DOCKER_IMAGE_MAIN_CPU_NAME_WITH_TAG \
--cuda_docker_uri=$DOCKER_IMAGE_MAIN_CUDA_NAME_WITH_TAG

In [ ]:
# The model gets saved to the path specified in the frozen task config.
print(f"Trained model saved to: {frozen_task_config.shared_config.trained_model_metadata.trained_model_uri}")

In [ ]:
%%bash
echo "JOB_NAME: $JOB_NAME"
echo "FROZEN_TASK_CONFIG_PATH: $FROZEN_TASK_CONFIG_PATH"
echo "GCS_RESOURCE_CONFIG_PATH: $GCS_RESOURCE_CONFIG_PATH"
echo "DOCKER_IMAGE_MAIN_CPU_NAME_WITH_TAG: $DOCKER_IMAGE_MAIN_CPU_NAME_WITH_TAG"
echo "DOCKER_IMAGE_MAIN_CUDA_NAME_WITH_TAG: $DOCKER_IMAGE_MAIN_CUDA_NAME_WITH_TAG"
echo "DOCKER_IMAGE_DATAFLOW_RUNTIME_NAME_WITH_TAG: $DOCKER_IMAGE_DATAFLOW_RUNTIME_NAME_WITH_TAG"
python -m gigl.src.inference.inferencer \
--job_name=$JOB_NAME \
--task_config_uri=$FROZEN_TASK_CONFIG_PATH \
--resource_config_uri=$GCS_RESOURCE_CONFIG_PATH \
--cpu_docker_uri=$DOCKER_IMAGE_MAIN_CPU_NAME_WITH_TAG \
--cuda_docker_uri=$DOCKER_IMAGE_MAIN_CUDA_NAME_WITH_TAG \
--custom_worker_image_uri=$DOCKER_IMAGE_DATAFLOW_RUNTIME_NAME_WITH_TAG

In [ ]:
# Looking at inference results

from gigl.src.common.utils.bq import BqUtils

bq_emb_out_table = frozen_task_config.shared_config.inference_metadata.node_type_to_inferencer_output_info_map[frozen_task_config.graph_metadata_pb_wrapper.homogeneous_node_type].embeddings_path
print(f"Embeddings should be successfully stored in the following location: {bq_emb_out_table}")

bq_utils = BqUtils(project=resource_config.project)
query = f"SELECT * FROM {bq_emb_out_table} LIMIT 5"
result = list(bq_utils.run_query(query=query, labels={}))

print(f"Query result: {result}")